# Sales CF-Based Recommender System

There are two variations of collaborative filtering recommender systems. There is a user-based recommender system and an item-based recommender system. They both recommend an item based on previous opinions of that item, or previous actions taken by the user regarding similar items.



![User-Item Matrix](userRatings.png)

# User Based Collaborative Filtering

The goal of user-based collaborative filtering is to predict the rating of $i_n$ given the previous actions of one user, $u_m$. In user-based collaborative filtering, we find $k$ candidates of other users that are similar to $u_m$. To find similarity, we can use 

$\begin{align}
\textbf{Cosine Similarity:} \\
cos \theta &= \frac{A\cdot B}{|A||B|}
= \frac{\sum_{i=1}^n A_iB_i}{\sqrt{\sum_{i=1}^nA^2_i}\sqrt{\sum_{i=1}^nB^2_i}}
\end{align}$




$\textbf{Pearson Correlation:}$
$\begin{align}
r &= \frac{\sum_{i=1}^n (x_i-\bar{x})(y_i-\bar{y})}{\sqrt{\sum_{i=1}^n (x_i-\bar{x})^2}\sqrt{\sum_{i=1}^n (y_i-\bar{y})^2}}\\
\end{align}$


There is also a method of using K-nearest-neighbors to find the most similar candidate users, defined by $$p_{a,i} = \bar{r}_a+\frac{\sum_{u \in K}(r_u, i - \bar{r}_u) \times w_{a,u}}{\sum_{u \in K}w_{a,u}},$$
where $p_{a,1}$ is the prediction for target user $u_m$ at item $i_n$, $w_{a,u}$ is the similarity between the users, and K is the neighborhood of similar users. 

# Item-based Collaborative Filtering

The goal of item-based collaborative filtering is to predict ratings for a specific item $i_n$ based on ratings of similar previous items rated by all possible users. 

The rating for target item $i_n$ for active user $u_m$ can be found using

$\textbf{Weighted Average}$ $$p_{a,i} = \frac{\sum_{j \in K} r_{a,j}w_{i,j}}{\sum_{j \in K}|W_{i,j}|}$$
where $K$ is the neighborhood of most similar items rated by user $u_m$ and $w(i,j)$ is the similarity between items $i$ and $j$. 

$\textbf{Adjusted Cosine Similarity}$ $$\textbf{sim}_ij = \frac{\sum_{u \in U}(R_{u,i}-\bar{R}_u)(R_{u,j}-\bar{R}_u)}{\sqrt{\sum_{u \in U}(R_{u,i}-\bar{R}_u)^2}\sqrt{\sum_{u \in U}(R_{u,j}-\bar{R}_u)^2}}$$

# Implementation

In this program, I am building a collaborative filtering model for recommending product items. 



*   Input: User -  Customer ID
*   Output: Ranked list of items (product ID) that the user most likely wants to buy 



In [3]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split

import sys

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. Load data
Two datasets are used in this exercise, which can be found in `data` folder: 
* `recommend_1.csv` consisting of a list of 1000 customer IDs to recommend as output
* `trx_data.csv` consisting of user transactions

The format is as follows.

In [ ]:
customers = pd.read_csv('/recommend_1.csv')
transactions = pd.read_csv('/trx_data.csv')

In [6]:
print(customers.shape)
customers.head()

(1000, 1)


,customerId
0,1553
1,20400
2,19750
3,6334
4,27773


In [7]:
print(transactions.shape)
transactions.head()

(62483, 2)


,customerId,products
0,0,20
1,1,2|2|23|68|68|111|29|86|107|152
2,2,111|107|29|11|11|11|33|23
3,3,164|227
4,5,2|2


## 3. Data preparation
* Our goal here is to break down each list of items in the `products` column into rows and count the number of products bought by a user

In [8]:
# example 1: split product items
transactions['products'] = transactions['products'].apply(lambda x: [int(i) for i in x.split('|')])
transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index()

,customerId,0,1,2,3,4,5,6,7,8,9
0,0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2.0,2.0,23.0,68.0,68.0,111.0,29.0,86.0,107.0,152.0


In [9]:
# example 2: organize a given table into a dataframe with customerId, single productId, and purchase count
pd.melt(transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,customerId,productId,purchase_count
0,0,20.0,1
1,1,2.0,2
2,1,23.0,1
3,1,29.0,1
4,1,68.0,2
5,1,86.0,1
6,1,107.0,1
7,1,111.0,1
8,1,152.0,1


### 3.1. Create data with user, item, and target field
* This table will be an input for our modeling later
    * In this case, our user is `customerId`, `productId`, and `purchase_count`

In [10]:
s=time.time()

data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

Execution time: 0.28 minutes


In [11]:
print(data.shape)
data.head()

(133585, 3)


,customerId,productId,purchase_count
0,0,1,2
1,0,13,1
2,0,19,3
3,0,20,1
4,0,31,2


### 3.2. Create dummy
* Dummy for marking whether a customer bought that item or not.
* If one buys an item, then `purchase_dummy` are marked as 1
* Why create a dummy instead of normalizing it, you ask?
    * Normalizing the purchase count, say by each user, would not work because customers may have different buying frequency don't have the same taste
    * However, we can normalize items by purchase frequency across all users, which is done in section 3.3. below.

In [ ]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

In [ ]:
data_dummy = create_data_dummy(data)

### 3.3. Normalize item values across users
* To do this, we normalize purchase frequency of each item across users by first creating a user-item matrix as follows

In [14]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix.head()

productId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
(df_matrix.shape)

(24429, 300)

In [16]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head()

(24429, 300)


productId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.142857,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.571429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.022222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# create a table for input to the modeling

d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(133585, 3)


,customerId,productId,scaled_purchase_freq
9,9,0,0.133333
25,25,0,0.133333
32,33,0,0.133333
35,36,0,0.133333
43,44,0,0.133333


#### Define a function for normalizing data

In [ ]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

* We can normalize the their purchase history, from 0-1 (with 1 being the most number of purchase for an item and 0 being 0 purchase count for that item).

## 4. Split train and test set
* Splitting the data into training and testing sets is an important part of evaluating predictive modeling, in this case a collaborative filtering model. Typically, we use a larger portion of the data for training and a smaller portion for testing. 
* We use 80:20 ratio for our train-test set size.
* Our training portion will be used to develop a predictive model, while the other to evaluate the model's performance.
* Now that we have three datasets with purchase counts, purchase dummy, and scaled purchase counts, we would like to split each.

In [19]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

(106868, 3) (26717, 3)


In [ ]:
# Using turicreate library, we convert dataframe to SFrame - this will be useful in the modeling part

train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [21]:
train_data

customerId,productId,purchase_count
843,140,1
28348,108,1
26980,87,3
466,162,1
15286,15,2
8599,2,2
7519,2,2
24574,87,1
3961,82,1
10954,171,1


In [22]:
test_data

customerId,productId,purchase_count
1750,146,2
2870,76,1
21091,4,2
9881,2,1
10807,189,1
8564,274,1
13417,1,2
25794,228,1
21384,228,2
15982,175,1


#### Define a `split_data` function for splitting data to training and test set

In [ ]:
# We can define a function for this step as follows

def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [ ]:
# lets try with both dummy table and scaled/normalized purchase table

train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

## 5. Baseline Model
Before running a more complicated approach such as collaborative filtering, we would like to use a baseline model to compare and evaluate models. Since baseline typically uses a very simple approach, techniques used beyond this approach should be chosen if they show relatively better accuracy and complexity.

### 5.1. Using a Popularity model as a baseline
* The popularity model takes the most popular items for recommendation. These items are products with the highest number of sells across customers.
* We use `turicreate` library for running and evaluating both baseline and collaborative filtering models below
* Training data is used for model selection

#### Using purchase counts

In [ ]:
# variables to define field names
user_id = 'customerId'
item_id = 'productId'
target = 'purchase_count'
users_to_recommend = list(transactions[user_id])
n_rec = 10 # number of items to recommend
n_display = 30

In [27]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 106868 observations with 23382 users and 300 items.

Data prepared in: 0.170297s

106868 observations to process; with 300 unique items.

In [28]:
# Get recommendations for a list of users to recommend (from customers file)
# Printed below is head / top 30 rows for first 3 customers with 10 recommendations each

popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/62483 queries. users per second: 114508

recommendations finished on 2000/62483 queries. users per second: 120664

recommendations finished on 3000/62483 queries. users per second: 119412

recommendations finished on 4000/62483 queries. users per second: 122380

recommendations finished on 5000/62483 queries. users per second: 121681

...

recommendations finished on 59000/62483 queries. users per second: 143431

recommendations finished on 60000/62483 queries. users per second: 143145

recommendations finished on 61000/62483 queries. users per second: 141939

recommendations finished on 62000/62483 queries. users per second: 139854

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|     0      |    248    | 3.272727272727273  |  1   |
|     0      |    132    | 3.1724137931034484 |  2   |
|     0      |     37    | 3.032258064516129  |  3   |
|     0      |     34    | 2.9959016393442623 |  4   |
|     0      |     0     | 2.992079207920792  |  5   |
|     0      |     3     | 2.800429184549356  |  6   |
|     0      |    110    | 2.7329545454545454 |  7   |
|     0      |     27    | 2.699248120300752  |  8   |
|     0      |    230    | 2.676258992805755  |  9   |
|     0      |     10    | 2.6516516516516515 |  10  |
|     1      |    248    | 3.272727272727273  |  1   |
|     1      |    132    | 3.1724137931034484 |  2   |
|     1      |     37    | 3.032258064516129  |  3   |
|     1      |     34    | 2.9959016393442623 |  4   |
|     1      |     0     | 2.992079207920792  |  5   |
|     1   

#### Define a `model` function for model selection

In [ ]:
# Since turicreate is very accessible library, we can define a model selection function as below

def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [ ]:
# variables to define field names
# constant variables include:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to print the head / first few rows in a defined dataset

#### Using purchase dummy

In [31]:
# these variables will change accordingly
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23305 users and 300 items.

Data prepared in: 0.154944s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 111123

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |    259    |  1.0  |  1   |
|    1553    |     44    |  1.0  |  2   |
|    1553    |    298    |  1.0  |  3   |
|    1553    |     45    |  1.0  |  4   |
|    1553    |    249    |  1.0  |  5   |
|    1553    |    106    |  1.0  |  6   |
|    1553    |     15    |  1.0  |  7   |
|    1553    |     3     |  1.0  |  8   |
|    1553    |     49    |  1.0  |  9   |
|    1553    |     11    |  1.0  |  10  |
|   20400    |     44    |  1.0  |  1   |
|   20400    |    298    |  1.0  |  2   |
|   20400    |     45    |  1.0  |  3   |
|   20400    |    249    |  1.0  |  4   |
|   20400    |    106    |  1.0  |  5   |
|   20400    |     15    |  1.0  |  6   |
|   20400    |     3     |  1.0  |  7   |
|   20400    |     49    |  1.0  |  8   |
|   20400    |    193    |  1.0  |  9   |
|   20400    |     11    |  1.0  |  10  |
|   19750    |     44    |  1.0  |

#### Using normalized purchase count

In [32]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23357 users and 300 items.

Data prepared in: 0.177034s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 99920.1

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.7567567567567568 |  1   |
|    1553    |    247    |  0.3433583959899749 |  2   |
|    1553    |    230    |  0.3319148936170206 |  3   |
|    1553    |    125    | 0.25611510791366876 |  4   |
|    1553    |    294    | 0.24806201550387563 |  5   |
|    1553    |     72    | 0.23844537815126052 |  6   |
|    1553    |    155    | 0.23333333333333325 |  7   |
|    1553    |    248    | 0.22965116279069767 |  8   |
|    1553    |    204    |  0.2282608695652172 |  9   |
|    1553    |    165    |  0.2265193370165746 |  10  |
|   20400    |    226    |  0.7567567567567568 |  1   |
|   20400    |    247    |  0.3433583959899749 |  2   |
|   20400    |    230    |  0.3319148936170206 |  3   |
|   20400    |    125    | 0.25611510791366876 |  4   |
|   20400    |    294    | 0.24806201550387563 |

#### Notes
* Once we created the model, we predicted the recommendation items using scores by popularity. As you can tell for each model results above, the rows show the first 30 records from 1000 users with 10 recommendations. These 30 records include 3 users and their recommended items, along with score and descending ranks. 
* In the result, although different models have different recommendation list, each user is recommended the same list of 10 items. This is because popularity is calculated by taking the most popular items across all users.
* If a grouping example below, products 132, 248, 37, and 34 are the most popular (best-selling) across customers. Using their purchase counts divided by the number of customers, we see that these products are at least bought 3 times on average in the training set of transactions (same as the first popularity measure on `purchase_count` variable)

In [33]:
train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
248    3.272727
132    3.172414
37     3.032258
34     2.995902
0      2.992079
3      2.800429
110    2.732955
27     2.699248
230    2.676259
10     2.651652
32     2.603774
58     2.563218
226    2.514493
245    2.456790
129    2.443787
173    2.439024
252    2.403509
68     2.402490
91     2.384298
41     2.354232
Name: purchase_count, dtype: float64

#### Using purchase count

In [34]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23382 users and 300 items.

Data prepared in: 0.169002s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.07754s

recommendations finished on 1000/1000 queries. users per second: 33992.8

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.07160244882106781  |  1   |
|    1553    |     35    |  0.0693587213754654  |  2   |
|    1553    |     1     |  0.0691552609205246  |  3   |
|    1553    |     61    | 0.06251166760921478  |  4   |
|    1553    |     21    | 0.04497094452381134  |  5   |
|    1553    |     76    | 0.04215094447135925  |  6   |
|    1553    |     0     | 0.03977116942405701  |  7   |
|    1553    |    269    | 0.037718966603279114 |  8   |
|    1553    |     8     | 0.03595167398452759  |  9   |
|    1553    |     5     | 0.03342823684215546  |  10  |
|   20400    |    122    | 0.045564889907836914 |  1   |
|   20400    |    215    | 0.04000645875930786  |  2   |
|   20400    |     6     | 0.03698962926864624  |  3   |
|   20400    |     1     | 0.036445021629333496 |  4   |
|   20400    |     54    | 0.03

#### Using purchase dummy

In [35]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23305 users and 300 items.

Data prepared in: 0.166418s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.101176s

recommendations finished on 1000/1000 queries. users per second: 57816.8

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     |  0.0987391710281372  |  1   |
|    1553    |     35    | 0.08719536066055297  |  2   |
|    1553    |     1     | 0.08481138944625854  |  3   |
|    1553    |     5     | 0.06987897157669068  |  4   |
|    1553    |     21    | 0.062110412120819095 |  5   |
|    1553    |     33    | 0.05326050519943237  |  6   |
|    1553    |     17    | 0.050858259201049805 |  7   |
|    1553    |     8     | 0.04967168569564819  |  8   |
|    1553    |    105    | 0.049265813827514646 |  9   |
|    1553    |     15    |  0.0440105676651001  |  10  |
|   20400    |     26    | 0.04677283763885498  |  1   |
|   20400    |     54    | 0.04296356439590454  |  2   |
|   20400    |    229    | 0.04230165481567383  |  3   |
|   20400    |     1     | 0.042143821716308594 |  4   |
|   20400    |    220    | 0.04

#### Using normalized purchase count

In [36]:
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23357 users and 300 items.

Data prepared in: 0.154898s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.08301s

recommendations finished on 1000/1000 queries. users per second: 44175.5

+------------+-----------+-----------------------+------+
| customerId | productId |         score         | rank |
+------------+-----------+-----------------------+------+
|    1553    |     25    |          0.0          |  1   |
|    1553    |    180    |          0.0          |  2   |
|    1553    |    228    |          0.0          |  3   |
|    1553    |     42    |          0.0          |  4   |
|    1553    |    288    |          0.0          |  5   |
|    1553    |     1     |          0.0          |  6   |
|    1553    |    205    |          0.0          |  7   |
|    1553    |     19    |          0.0          |  8   |
|    1553    |    133    |          0.0          |  9   |
|    1553    |     2     |          0.0          |  10  |
|   20400    |     1     | 0.0037458515167236328 |  1   |
|   20400    |     2     | 0.0036238288879394532 |  2   |
|   20400    |     8     |  0.002835109233856201 |  3   |
|   20400    |     38    | 0.0023179936408996584 |  4   |
|   20400    |

#### Using purchase count

In [37]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23382 users and 300 items.

Data prepared in: 0.154145s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.095617s

recommendations finished on 1000/1000 queries. users per second: 47321.6

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|    1553    |    248    | 3.269303671338341  |  1   |
|    1553    |    132    | 3.172413793103448  |  2   |
|    1553    |     37    | 3.0322580645161303 |  3   |
|    1553    |     34    | 2.9947963457127105 |  4   |
|    1553    |     0     | 2.9891616797683294 |  5   |
|    1553    |     3     | 2.7993184668951274 |  6   |
|    1553    |    110    | 2.7272471324963994 |  7   |
|    1553    |     27    |  2.69852878164528  |  8   |
|    1553    |    230    | 2.673649873986517  |  9   |
|    1553    |     10    | 2.6516516516516515 |  10  |
|   20400    |    248    | 3.2727272727272725 |  1   |
|   20400    |    132    | 3.172413793103448  |  2   |
|   20400    |     37    | 3.029654354818407  |  3   |
|   20400    |     34    | 2.995901639344264  |  4   |
|   20400    |     0     | 2.992079207920795  |  5   |
|   20400 

#### Using purchase dummy

In [38]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23305 users and 300 items.

Data prepared in: 0.203718s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.087161s

recommendations finished on 1000/1000 queries. users per second: 60834.7

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |    259    |  0.0  |  1   |
|    1553    |     44    |  0.0  |  2   |
|    1553    |    298    |  0.0  |  3   |
|    1553    |     45    |  0.0  |  4   |
|    1553    |    249    |  0.0  |  5   |
|    1553    |    106    |  0.0  |  6   |
|    1553    |     15    |  0.0  |  7   |
|    1553    |     3     |  0.0  |  8   |
|    1553    |     49    |  0.0  |  9   |
|    1553    |     11    |  0.0  |  10  |
|   20400    |     44    |  0.0  |  1   |
|   20400    |    298    |  0.0  |  2   |
|   20400    |     45    |  0.0  |  3   |
|   20400    |    249    |  0.0  |  4   |
|   20400    |    106    |  0.0  |  5   |
|   20400    |     15    |  0.0  |  6   |
|   20400    |     3     |  0.0  |  7   |
|   20400    |     49    |  0.0  |  8   |
|   20400    |    193    |  0.0  |  9   |
|   20400    |     11    |  0.0  |  10  |
|   19750    |     44    |  0.0  |

#### Using normalized purchase count

In [39]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23357 users and 300 items.

Data prepared in: 0.148235s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.116343s

recommendations finished on 1000/1000 queries. users per second: 61527.1

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.7567567567567564 |  1   |
|    1553    |    247    | 0.34335839598997503 |  2   |
|    1553    |    230    |  0.331357133219428  |  3   |
|    1553    |    125    | 0.25611510791366904 |  4   |
|    1553    |    294    |  0.248062015503876  |  5   |
|    1553    |     72    | 0.23844537815126055 |  6   |
|    1553    |    155    | 0.23333333333333334 |  7   |
|    1553    |    248    | 0.22965116279069767 |  8   |
|    1553    |    204    | 0.22826086956521746 |  9   |
|    1553    |    165    |  0.2265193370165745 |  10  |
|   20400    |    226    |  0.7567425398568846 |  1   |
|   20400    |    247    |  0.3433559915386048 |  2   |
|   20400    |    230    |  0.3319112923043839 |  3   |
|   20400    |    125    | 0.25610130824631067 |  4   |
|   20400    |    294    | 0.24805311653041104 |

#### Note
* In collaborative filtering above, we used two approaches: cosine and pearson distance. We also got to apply them to three training datasets with normal counts, dummy, or normalized counts of items purchase.
* We can see that the recommendations are different for each user. This suggests that personalization does exist. 
* But how good is this model compared to the baseline, and to each other? We need some means of evaluating a recommendation engine. Lets focus on that in the next section.

## 7. Model Evaluation
For evaluating recommendation engines, we can use the concept of precision-recall.

* RMSE (Root Mean Squared Errors)
    * Measures the error of predicted values
    * Lesser the RMSE value, better the recommendations
* Recall
    * What percentage of products that a user buys are actually recommended?
    * If a customer buys 5 products and the recommendation decided to show 3 of them, then the recall is 0.6
* Precision
    * Out of all the recommended items, how many the user actually liked?
    * If 5 products were recommended to the customer out of which he buys 4 of them, then precision is 0.8
    
Lets compare all the models we have built based on precision-recall characteristics:

In [ ]:
# create initial callable variables

models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]

names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

#### Models on purchase counts

In [41]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/13856 queries. users per second: 56912

recommendations finished on 2000/13856 queries. users per second: 49164.2

recommendations finished on 3000/13856 queries. users per second: 51457.1

recommendations finished on 4000/13856 queries. users per second: 53611.4

...

recommendations finished on 10000/13856 queries. users per second: 40686.1

recommendations finished on 11000/13856 queries. users per second: 41140.6

recommendations finished on 12000/13856 queries. users per second: 41523.4

recommendations finished on 13000/13856 queries. users per second: 41480.4


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    | 0.0010825635103926085 | 0.0005335491586934974 |
|   2    | 0.0009021362586605125 | 0.0009665745628505469 |
|   3    |  0.002670323325635082 | 0.0038991188276696386 |
|   4    |  0.004294168591224017 |  0.008912776884233298 |
|   5    |  0.006899538106235589 |  0.01870050864178426  |
|   6    |  0.007349403387220945 |  0.023637120616863343 |
|   7    |  0.006639722863741344 |  0.02480328208722529  |
|   8    |  0.006125505196304863 |  0.02626071901214909  |
|   9    |  0.005845842956120112 |  0.028242153907123403 |
|   10   |  0.005795323325635103 |  0.03137364069641161  |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0909592024577166

Per User RMSE (best)
+------------+-----------------------+------

#### Models on purchase dummy

In [42]:
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)

PROGRESS: Evaluate model Popularity Model on Purchase Dummy


recommendations finished on 1000/13944 queries. users per second: 61087.4

recommendations finished on 2000/13944 queries. users per second: 58508.6

recommendations finished on 3000/13944 queries. users per second: 62862.8

recommendations finished on 4000/13944 queries. users per second: 65211.4

...

recommendations finished on 10000/13944 queries. users per second: 54208.5

recommendations finished on 11000/13944 queries. users per second: 55393.3

recommendations finished on 12000/13944 queries. users per second: 55687.3

recommendations finished on 13000/13944 queries. users per second: 52783.1


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    |  0.005306942053929974 |  0.002646640985711559 |
|   2    |  0.006024096385542138 | 0.0064205233632944414 |
|   3    |  0.006095811818703401 |  0.009624633629582022 |
|   4    | 0.0057730923694779305 |  0.012163647135678154 |
|   5    |  0.005464716006884674 |  0.014559198729809803 |
|   6    |  0.00505593803786575  |  0.01575180930817772  |
|   7    |  0.00479468895992131  |  0.017344872654792946 |
|   8    |  0.004751147446930579 |  0.01943765679899696  |
|   9    | 0.0047571237330273405 |  0.021808352475953537 |
|   10   | 0.0047834193918531295 |  0.024014600680480627 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0

Per User RMSE (best)
+------------+------+-------+
| customerId | rmse | count |

#### Models on normalized purchase frequency

In [43]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts


recommendations finished on 1000/13902 queries. users per second: 52606.7

recommendations finished on 2000/13902 queries. users per second: 51729

recommendations finished on 3000/13902 queries. users per second: 55451.8

recommendations finished on 4000/13902 queries. users per second: 57327.1

...

recommendations finished on 10000/13902 queries. users per second: 41603.2

recommendations finished on 11000/13902 queries. users per second: 42073.9

recommendations finished on 12000/13902 queries. users per second: 42291.2

recommendations finished on 13000/13902 queries. users per second: 41144.2


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    |  0.002301827075240963 | 0.0013465288767403607 |
|   2    | 0.0020860307869371287 | 0.0021785434549784956 |
|   3    | 0.0020140986908358583 |  0.003324661519525555 |
|   4    | 0.0020680477629118057 |  0.004630174569751607 |
|   5    |  0.002229894979139697 |  0.006361556294471809 |
|   6    | 0.0033208651033424493 |  0.011241537170164885 |
|   7    |  0.003000596008796267 |  0.011924492460370958 |
|   8    |  0.002751402675873962 |  0.012346494090831779 |
|   9    | 0.0026534951006250145 |  0.013334068177694901 |
|   10   |  0.002762192490289189 |  0.015173848308368414 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.13211031619884706

Per User RMSE (best)
+------------+------------------------+----

## 8. Model Selection
### 8.1. Evaluation summary
* Based on RMSE


    1. Popularity on purchase counts: 1.1111750034210488
    2. Cosine similarity on purchase counts: 1.9230643981653215
    3. Pearson similarity on purchase counts: 1.9231102838192284
    
    4. Popularity on purchase dummy: 0.9697374361161925
    5. Cosine similarity on purchase dummy: 0.9697509978436404
    6. Pearson similarity on purchase dummy: 0.9697745320187097
    
    7. Popularity on scaled purchase counts: 0.16230660626840343
    8. Cosine similarity on scaled purchase counts: 0.16229800354111104
    9. Pearson similarity on scaled purchase counts: 0.1622982668334026
    
* Based on Precision and Recall
![](../images/model_comparisons.png)


#### Notes

* Popularity v. Collaborative Filtering: We can see that the collaborative filtering algorithms work better than popularity model for purchase counts. Indeed, popularity model doesn’t give any personalizations as it only gives the same list of recommended items to every user.
* Precision and recall: Looking at the summary above, we see that the precision and recall for Purchase Counts > Purchase Dummy > Normalized Purchase Counts. However, because the recommendation scores for the normalized purchase data is zero and constant, we choose the dummy. In fact, the RMSE isn’t much different between models on the dummy and those on the normalized data.
* RMSE: Since RMSE is higher using pearson distance thancosine, we would choose model the smaller mean squared errors, which in this case would be cosine.
Therefore, we select the Cosine similarity on Purchase Dummy approach as our final model.

## 8. Final Output
* In this step, we would like to manipulate format for recommendation output to one we can export to csv, and also a function that will return recommendation list given a customer ID.
* We need to first rerun the model using the whole dataset, as we came to a final model using train data and evaluated with test set.

In [44]:
users_to_recommend = list(customers[user_id])

final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy', 
                                            similarity_type='cosine')

recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 133585 observations with 24429 users and 300 items.

Data prepared in: 0.168159s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.099238s

recommendations finished on 1000/1000 queries. users per second: 66800.3

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     1     | 0.10348175764083863  |  1   |
|    1553    |     2     |  0.0934672474861145  |  2   |
|    1553    |     35    |  0.0845762014389038  |  3   |
|    1553    |     33    |  0.0668614387512207  |  4   |
|    1553    |     61    | 0.06512556076049805  |  5   |
|    1553    |     15    | 0.06476415395736694  |  6   |
|    1553    |     11    | 0.05467898845672607  |  7   |
|    1553    |     5     | 0.05406981706619263  |  8   |
|    1553    |     36    | 0.05048650503158569  |  9   |
|    1553    |     13    | 0.04985467195510864  |  10  |
|   20400    |     26    | 0.05812269449234009  |  1   |
|   20400    |     6     | 0.05361741781234741  |  2   |
|   20400    |    113    | 0.05312788486480713  |  3   |
|   20400    |     1     | 0.05210459232330322  |  4   |
|   20400    |     15    | 0.04

### 8.2. Customer recommendation function

In [ ]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [53]:
customer_recomendation(4)

recommendedProducts    2|1|36|13|216|61|20|33|25|157
Name: 4, dtype: object

In [54]:
customer_recomendation(21)

recommendedProducts    38|36|48|79|1|2|15|13|25|20
Name: 21, dtype: object

## Summary
In this exercise, we were able to traverse a step-by-step process for making recommendations to customers. We used Collaborative Filtering approaches with `cosine` and `pearson` measure and compare the models with our baseline popularity model. We also prepared three sets of data that include regular buying count, buying dummy, as well as normalized purchase frequency as our target variable. Using RMSE, precision and recall, we evaluated our models and observed the impact of personalization. Finally, we selected the Cosine approach in dummy purchase data. 